In [1]:
import numpy as np
import scipy
import os
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(0)

In [2]:
# Read Files
def load_data(dir_path):
    '''
        Function to Read Data from mat Files
    '''
    
    mat_files = []
    for m in os.listdir(dir_path):
        if m.endswith('.mat'):
            mat_files.append(m)
    
    data = {}
    for sbj, file in enumerate(mat_files):
        file_path = os.path.join(dir_path, file)
        mat_data = scipy.io.loadmat(file_path)
        data[sbj] = (mat_data)

    return data

In [3]:
def FilterBandPass(CNT, Wn, Fs):    
    Wn = np.array([Wn[0] / (Fs/2), Wn[1] / (Fs/2)])
    
    # Filter Design
    order = 3
    type = 'bandpass'
    b, a = scipy.signal.butter(order, Wn, btype = 'bandpass')
    
    # Apply Filter
    CNT = scipy.signal.filtfilt(b, a, CNT, axis = 0)
    
    return CNT

In [4]:
# CAR Filter for Spatial Filtering
def CARFilter(Data):
    M = np.mean(Data, axis = 1)
    
    for i in range(Data.shape[1]):
        Data[:, i] = Data[:, i] - M
    
    return Data

In [5]:
def RCSP(Train1, Train2, m, Alpha, Beta, Gamma, Gc1, Gc2, Sc1, Sc2, SbjNum):
    Rh = 0
    for i in range(Train1.shape[2]):
        Data = Train1[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rh += Data.T @ Data / np.trace(Data.T @ Data)

    Rf = 0
    for i in range(Train2.shape[2]):
        Data = Train2[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rf += Data.T @ Data / np.trace(Data.T @ Data)


    # Normalize Cov Matrix
    Rf /= Train2.shape[2]
    Rh /= Train1.shape[2]
    
    C_hat1 = (1 - Beta) * Sc1[SbjNum] * Rh + Beta * Gc1[:, :, SbjNum]
    C_hat2 = (1 - Beta) * Sc2[SbjNum] * Rf + Beta * Gc2[:, :, SbjNum]
    
    C_Not1 = (1 - Gamma) * C_hat1 + Gamma * np.identity(Rf.shape[0])
    C_Not2 = (1 - Gamma) * C_hat2 + Gamma * np.identity(Rf.shape[0])
    
    
    R1 = C_Not1 + Alpha * np.identity(Rh.shape[0])
    R2 = C_Not2 + Alpha * np.identity(Rh.shape[0])
    
    # Eigen Value Decomposition
    u1, v1 = scipy.linalg.eig(C_Not1, R2)
    # u = np.real(u)
    
    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u1)[::-1]  # Sort in descending order
    u1 = u1[sorted_indices]
    v1 = v1[:, sorted_indices]
    
    
    # Eigen Value Decomposition
    u2, v2 = scipy.linalg.eig(C_Not2, R1)
    # u = np.real(u)
    
    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u2)[::-1]  # Sort in descending order
    u2 = u2[sorted_indices]
    v2 = v2[:, sorted_indices]
    
    W = np.concatenate((v1[:, :m], v2[:, :m]), axis = 1)


    return W

In [6]:
def Cov(Train1, Train2):
    C1 = 0
    for i in range(Train1.shape[2]):
        Data = Train1[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        C1 += Data.T @ Data / np.trace(Data.T @ Data)

    C2 = 0
    for i in range(Train2.shape[2]):
        Data = Train2[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        C2 += Data.T @ Data / np.trace(Data.T @ Data)


    # Normalize Cov Matrix
    C1 /= Train1.shape[2]
    C2 /= Train2.shape[2]
    
    Ntr = [Train1.shape[2], Train2.shape[2]]
    
    return C1, C2, Ntr

In [7]:
dir_path = './Dataset'
data = load_data(dir_path)
np.random.seed(0)

AvailbaleDataKeys = data.keys()
# AvailbaleDataKeys = [6]

Ntr = []

for key in list(AvailbaleDataKeys):
    
    # Read Data
    cnt = data[key]['cnt']
    nfo = data[key]['nfo']
    mrk = data[key]['mrk']
    fs = nfo['fs'][0][0][0][0]
    
    ## Filtering Data
    cnt = FilterBandPass(cnt, [8, 30], fs)
    
    # Spatial Filtering
    cnt = CARFilter(cnt)
    
    # Trial Seperation
    Ltr = 4 * fs
    Pos = mrk['pos'][0][0][0]
    Group = mrk['y'][0][0][0]
    
    data1 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    data2 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    c1 = 0
    c2 = 0
    
    for i in range(len(Pos)):
        Idx = range(Pos[i], Pos[i] + Ltr)
        Trial = cnt[Idx, :]
        
        if Group[i] == 1:
            data1[:, :, c1] = Trial
            c1 += 1
        elif Group[i] == -1:
            data2[:, :, c2] = Trial
            c2 += 1

            
            
    cov1, cov2, ntr = Cov(data1, data2)
    
    if key == 0:
        Cov1 = cov1
        Cov2 = cov2
    else:
        Cov1 = np.dstack((Cov1, cov1))
        Cov2 = np.dstack((Cov2, cov2))
    
    Ntr.append(ntr)

Ntr = np.array(Ntr)

In [8]:
Nt1 = np.sum(Ntr[:, 0])
Nt2 = np.sum(Ntr[:, 1])

Gc1 = np.zeros(Cov1.shape)
Gc2 = np.zeros(Cov2.shape)

Sc1 = np.zeros(Cov1.shape[2])
Sc2 = np.zeros(Cov2.shape[2])

for i in range(Cov1.shape[2]):
    Idx = np.arange(Cov1.shape[2])
    Idx = np.delete(Idx, i)

    gc1 = 0
    gc2 = 0
    
    for j in Idx:
        C1 = Cov1[:, :, j]
        C2 = Cov2[:, :, j]
        
        N1 = Ntr[j, 0]
        N2 = Ntr[j, 1]
        
        gc1 = gc1 + (N1 / Nt1) * C1 
        gc2 = gc2 + (N2 / Nt2) * C2
        
    
    Gc1[:, :, i] = gc1
    Gc2[:, :, i] = gc2
    Sc1[i] = Ntr[i, 0] / Nt1
    Sc2[i] = Ntr[i, 1] / Nt2

In [9]:
dir_path = './Dataset'
data = load_data(dir_path)
np.random.seed(0)

AvailbaleDataKeys = data.keys()
# AvailbaleDataKeys = [6]

for key in list(AvailbaleDataKeys):
    
    # Read Data
    cnt = data[key]['cnt']
    nfo = data[key]['nfo']
    mrk = data[key]['mrk']
    fs = nfo['fs'][0][0][0][0]
    
    ## Filtering Data
    cnt = FilterBandPass(cnt, [8, 30], fs)
    
    # Spatial Filtering
    cnt = CARFilter(cnt)
    
    # Trial Seperation
    Ltr = 4 * fs
    Pos = mrk['pos'][0][0][0]
    Group = mrk['y'][0][0][0]
    
    data1 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    data2 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    c1 = 0
    c2 = 0
    
    for i in range(len(Pos)):
        Idx = range(Pos[i], Pos[i] + Ltr)
        Trial = cnt[Idx, :]
        
        if Group[i] == 1:
            data1[:, :, c1] = Trial
            c1 += 1
        elif Group[i] == -1:
            data2[:, :, c2] = Trial
            c2 += 1

    ## Train and Test Data Separation
    # Index Creation
    TestDiv = 0.2
    Idx = np.random.choice(data1.shape[2], size=int(TestDiv * data1.shape[2]), replace=False)
    TestIdx = np.zeros(data1.shape[2], dtype=bool)
    TestIdx[Idx] = True
    
    TrainIdx = ~TestIdx
    
    # Data Seperation
    Train1 = data1[:, :, TrainIdx]
    Train2 = data2[:, :, TrainIdx]
    
    Test1 = data1[:, :, TestIdx]
    Test2 = data2[:, :, TestIdx]
    
    # CSP Weight Matrix Creation
    M     = 2
    Alpha = 1e-4 
    Beta  = 1e-9
    Gamma = 1e-8
    
    W = RCSP(Train1, Train2, M, Alpha, Beta, Gamma, Gc1, Gc2, Sc1, Sc2, key)

    # Train Feature Extraction
    Train = np.concatenate((Train1, Train2), axis = 2)    
    FeatureTrain = np.zeros((2 * M, Train.shape[2]))
    
    for i in range(Train.shape[2]):
        tmp = Train[:, :, i].T
        tmp = W.T @ tmp
        FeatureTrain[:, i] = np.var(tmp, axis = 1)
    
    # Test Feature Extraction
    Test = np.concatenate((Test1, Test2), axis = 2) 
    FeatureTest = np.zeros((2 * M, Test.shape[2]))
    
    for i in range(Test.shape[2]):
        tmp = Test[:, :, i].T
        tmp = W.T @ tmp
        FeatureTest[:, i] = np.var(tmp, axis = 1)
        
    # Label Creation
    LabelTrain = np.concatenate((np.ones(Train1.shape[2]), 2 * np.ones(Train2.shape[2])))
    LabelTest = np.concatenate((np.ones(Test1.shape[2]), 2 * np.ones(Test2.shape[2])))
    
    
    # Train and Test Classifier
    mdl = KNeighborsClassifier(n_neighbors = 5)
    # mdl = svm.SVC(kernel='linear', C = 1.0)
    
    mdl.fit(FeatureTrain.T, LabelTrain)
    LabelPredict = mdl.predict(FeatureTest.T)
    
    Acc = accuracy_score(LabelTest, LabelPredict)
    
    # Print Metrics
    print(f'Subject {key}')
    print("Accuracy : ", accuracy_score(LabelTest, LabelPredict))
    print("Confusion Matrix : \n", confusion_matrix(LabelTest, LabelPredict))
    print('-----------------------------------------------------------------')

Subject 0
Accuracy :  0.825
Confusion Matrix : 
 [[16  4]
 [ 3 17]]
-----------------------------------------------------------------
Subject 1
Accuracy :  0.675
Confusion Matrix : 
 [[16  4]
 [ 9 11]]
-----------------------------------------------------------------
Subject 2
Accuracy :  0.5
Confusion Matrix : 
 [[11  9]
 [11  9]]
-----------------------------------------------------------------
Subject 3
Accuracy :  0.6
Confusion Matrix : 
 [[11  9]
 [ 7 13]]
-----------------------------------------------------------------
Subject 4
Accuracy :  0.925
Confusion Matrix : 
 [[17  3]
 [ 0 20]]
-----------------------------------------------------------------
Subject 5
Accuracy :  0.825
Confusion Matrix : 
 [[15  5]
 [ 2 18]]
-----------------------------------------------------------------
Subject 6
Accuracy :  0.825
Confusion Matrix : 
 [[14  6]
 [ 1 19]]
-----------------------------------------------------------------


In [10]:
dir_path = './Dataset'
data = load_data(dir_path)
np.random.seed(0)

AvailbaleDataKeys = data.keys()
# AvailbaleDataKeys = [6]


for key in list(AvailbaleDataKeys):
    
    # Read Data
    cnt = data[key]['cnt']
    nfo = data[key]['nfo']
    mrk = data[key]['mrk']
    fs = nfo['fs'][0][0][0][0]
    
    ## Filtering Data
    cnt = FilterBandPass(cnt, [8, 30], fs)
    
    # Spatial Filtering
    cnt = CARFilter(cnt)
    
    # Trial Seperation
    Ltr = 4 * fs
    Pos = mrk['pos'][0][0][0]
    Group = mrk['y'][0][0][0]
    
    data1 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    data2 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    c1 = 0
    c2 = 0
    
    for i in range(len(Pos)):
        Idx = range(Pos[i], Pos[i] + Ltr)
        Trial = cnt[Idx, :]
        
        if Group[i] == 1:
            data1[:, :, c1] = Trial
            c1 += 1
        elif Group[i] == -1:
            data2[:, :, c2] = Trial
            c2 += 1

    ## Train and Test Data Separation
    # Index Creation
    TestDiv = 0.2
    Idx = np.random.choice(data1.shape[2], size=int(TestDiv * data1.shape[2]), replace=False)
    TestIdx = np.zeros(data1.shape[2], dtype=bool)
    TestIdx[Idx] = True
    
    TrainIdx = ~TestIdx
    
    # Data Seperation
    Train1 = data1[:, :, TrainIdx]
    Train2 = data2[:, :, TrainIdx]
    
    Test1 = data1[:, :, TestIdx]
    Test2 = data2[:, :, TestIdx]
    
    # CSP Weight Matrix Creation
    M = 2
    
    # Parameter Excellence Matrix Creation
    AlphaList = [0, 10e-10, 10e-9, 10e-8, 10e-7, 10e-6, 10e-5, 10e-4, 10e-3, 10e-2, 10e-1] 
    BetaList  = [0, 10e-10, 10e-9, 10e-8] 
    GammaList = [0, 10e-10, 10e-9]

    ParamEx = np.zeros((len(AlphaList) * len(BetaList) * len(GammaList), 7))
    
    c = 0
    for Alpha in AlphaList:
        for Beta in BetaList:
            for Gamma in GammaList:
                ParamEx[c, 0] = Alpha
                ParamEx[c, 1] = Beta
                ParamEx[c, 2] = Gamma
                
                W = RCSP(Train1, Train2, M, Alpha, Beta, Gamma, Gc1, Gc2, Sc1, Sc2, key)

                # Train Feature Extraction
                Train = np.concatenate((Train1, Train2), axis = 2)    
                FeatureTrain = np.zeros((2 * M, Train.shape[2]))

                for i in range(Train.shape[2]):
                    tmp = Train[:, :, i].T
                    tmp = W.T @ tmp
                    FeatureTrain[:, i] = np.var(tmp, axis = 1)

                # Test Feature Extraction
                Test = np.concatenate((Test1, Test2), axis = 2) 
                FeatureTest = np.zeros((2 * M, Test.shape[2]))

                for i in range(Test.shape[2]):
                    tmp = Test[:, :, i].T
                    tmp = W.T @ tmp
                    FeatureTest[:, i] = np.var(tmp, axis = 1)

                # Label Creation
                LabelTrain = np.concatenate((np.ones(Train1.shape[2]), 2 * np.ones(Train2.shape[2])))
                LabelTest = np.concatenate((np.ones(Test1.shape[2]), 2 * np.ones(Test2.shape[2])))


                # Train and Test Classifier
                mdl = KNeighborsClassifier(n_neighbors = 5)
                # mdl = svm.SVC(kernel='linear', C = 1.0)

                mdl.fit(FeatureTrain.T, LabelTrain)
                LabelPredict = mdl.predict(FeatureTest.T)

                Acc = accuracy_score(LabelTest, LabelPredict)
                ConfMat = confusion_matrix(LabelTest, LabelPredict)
                
                ParamEx[c, 3] = Acc
                ParamEx[c, 4] = (np.diag(ConfMat) / ConfMat.sum(axis=1)).prod()
                
                ParamEx[c, 5] = (np.diag(ConfMat) / ConfMat.sum(axis=1))[0]
                ParamEx[c, 6] = (np.diag(ConfMat) / ConfMat.sum(axis=1))[1]
                c += 1
    
    
    # Get the Index of Best Accuracy from the Last Column
    BestIdx   = np.argmax(ParamEx[:, 4])
    BestAlpha = ParamEx[BestIdx, 0]
    BestBeta  = ParamEx[BestIdx, 1]
    BestGamma = ParamEx[BestIdx, 2]
    BestAcc   = ParamEx[BestIdx, 3]
    C1BestAcc = ParamEx[BestIdx, 5]
    C2BestAcc = ParamEx[BestIdx, 6]

    # Print Metrics
    print(f'Subject: {key + 1}')
    print("Mean Accurcy of All Classes : ", BestAcc * 100)
    print(f"Parameters: Alpha = {BestAlpha:.2e}, Beta = {BestBeta:.2e}, Gamma = {BestGamma:.2e}")
    print(f"Class 1 Accuracy : {C1BestAcc * 100}")
    print(f"Class 2 Accuracy : {C2BestAcc * 100}")
    print('-----------------------------------------------------------------')

Subject: 1
Mean Accurcy of All Classes :  90.0
Parameters: Alpha = 0.00e+00, Beta = 0.00e+00, Gamma = 1.00e-09
Class 1 Accuracy : 90.0
Class 2 Accuracy : 90.0
-----------------------------------------------------------------
Subject: 2
Mean Accurcy of All Classes :  67.5
Parameters: Alpha = 1.00e-05, Beta = 0.00e+00, Gamma = 0.00e+00
Class 1 Accuracy : 70.0
Class 2 Accuracy : 65.0
-----------------------------------------------------------------
Subject: 3
Mean Accurcy of All Classes :  75.0
Parameters: Alpha = 0.00e+00, Beta = 1.00e-07, Gamma = 0.00e+00
Class 1 Accuracy : 65.0
Class 2 Accuracy : 85.0
-----------------------------------------------------------------
Subject: 4
Mean Accurcy of All Classes :  82.5
Parameters: Alpha = 0.00e+00, Beta = 1.00e-09, Gamma = 0.00e+00
Class 1 Accuracy : 75.0
Class 2 Accuracy : 90.0
-----------------------------------------------------------------
Subject: 5
Mean Accurcy of All Classes :  92.5
Parameters: Alpha = 1.00e-04, Beta = 0.00e+00, Gamma 